In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, RepeatVector, TimeDistributed, LayerNormalization
from sklearn.preprocessing import MinMaxScaler

# Fetch data from Yahoo Finance
data = yf.download("BTC-USD", start="2024-06-01", end="2024-12-01", interval="1h")

# Preprocess data
data.dropna(inplace=True)
data['Close'] = data['Close'].astype(float)

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Function to create dataset
def create_dataset(dataset, look_back=60):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 60
X, y = create_dataset(scaled_data, look_back)

# Reshape X to 3D array for LSTM (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Temporal Attention Layer (custom layer)
class TemporalAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(TemporalAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

    def build(self, input_shape):
        self.query_dense = Dense(self.d_model)
        self.key_dense = Dense(self.d_model)
        self.value_dense = Dense(self.d_model)

    def call(self, inputs):
        # Ensure inputs are cast to float32
        inputs = tf.cast(inputs, tf.float32)

        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # Split the last dimension into (num_heads, depth)
        query = tf.reshape(query, (tf.shape(query)[0], -1, self.num_heads, self.d_model // self.num_heads))
        key = tf.reshape(key, (tf.shape(key)[0], -1, self.num_heads, self.d_model // self.num_heads))
        value = tf.reshape(value, (tf.shape(value)[0], -1, self.num_heads, self.d_model // self.num_heads))

        # Scaled dot-product attention
        attention_scores = tf.matmul(query, key, transpose_b=True) / tf.sqrt(tf.cast(self.d_model, tf.float32))
        attention_weights = tf.nn.softmax(attention_scores, axis=-1)
        output = tf.matmul(attention_weights, value)

        # Concatenate the heads and reshape to original shape
        output = tf.reshape(output, (tf.shape(output)[0], -1, self.d_model))

        return output

# Model Architecture
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(look_back, 1)))
model.add(RepeatVector(48))  # Adjust the number of time steps (48 for 48 hours prediction)
model.add(TemporalAttention(50, 8))  # Temporal Attention Layer (d_model and num_heads are adjustable)
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mse', optimizer='adam')
model.summary()
# Model Training
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)

# Prepare the last 60 hours of data for prediction
last_60_hours = scaled_data[-60:]
X_test = last_60_hours.reshape((1, look_back, 1))

# Predict the next 48 hours
y_pred = model.predict(X_test)

# Inverse transform to get original price scale
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))  # Reshape the predictions before inverse transforming

print(y_pred)


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                        │ (None, 50)                  │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector_6 (RepeatVector)       │ (None, 48, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ temporal_attention_4                 │ (None, None, 50)            │           7,650 │
│ (TemporalAttention)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, None, 50)            │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, None, 1)             │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,301 (149.61 KB)

 Trainable params: 38,301 (149.61 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 13s 75ms/step - loss: 0.0217 - val_loss: 0.1711
Epoch 2/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - loss: 0.0078 - val_loss: 0.1926
Epoch 3/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 8s 71ms/step - loss: 0.0070 - val_loss: 0.2040
Epoch 4/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 10s 69ms/step - loss: 0.0072 - val_loss: 0.2092
Epoch 5/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 9s 84ms/step - loss: 0.0070 - val_loss: 0.2036
Epoch 6/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - loss: 0.0068 - val_loss: 0.1936
Epoch 7/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 10s 69ms/step - loss: 0.0065 - val_loss: 0.2119
Epoch 8/100
 52/109 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - loss: 0.0070